In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
from scipy.io import loadmat
from sklearn.model_selection import train_test_split

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [29]:
class Net(nn.Module):
    """Basic feedforward neural network with tanh activations."""
    def __init__(self, inp=2, out=1, hid=128, layers=3):
        super().__init__()
        net = []

        # Input layer
        net.append(nn.Linear(inp, hid))
        net.append(nn.Tanh())

        # Hidden layers
        for _ in range(layers - 1):
            net.append(nn.Linear(hid, hid))
            net.append(nn.Tanh())

        # Output layer
        net.append(nn.Linear(hid, out))
        self.net = nn.Sequential(*net)

        # Weight initialization for stable training
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        """Forward pass through the network"""
        return self.net(x)

In [30]:
def burgers_pde(model, x, nu):
    """
    model : neural network representing u(x,t)
    x     : input tensor with columns [x, t]
    nu    : viscosity constant
    """
    x.requires_grad_(True)                     # enable autograd for PDE derivatives
    u = model(x)                               # predicted u(x,t)
    grads = autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    u_x, u_t = grads[:, 0:1], grads[:, 1:2]    # first derivatives wrt x and t
    u_xx = autograd.grad(u_x, x, torch.ones_like(u_x), create_graph=True)[0][:, 0:1]  # second derivative wrt x
    f = u_t + u * u_x - nu * u_xx              # Burgers PDE residual
    return f

In [31]:
class PINN:
    """Trains network using data + PDE physics residual"""
    def __init__(self, pde_fn, inp, out, nu=0.01/np.pi, lam_data=1.0, lam_phys=0.1):
        self.net = Net(inp, out).to(device)       # neural network
        self.pde_fn = pde_fn                      # PDE function (burgers_pde)
        self.nu = nu                              # viscosity (used in Burgers')
        self.lam_d = lam_data                     # weight for data loss
        self.lam_p = lam_phys                     # weight for physics loss
        self.loss_fn = nn.MSELoss()               # mean squared error
        self.opt = torch.optim.Adam(self.net.parameters(), lr=1e-3)  # optimizer

    def train(self, x_train, u_train, x_val, u_val, a_fn=None, epochs=5000, colloc=2000):
        """Main training loop"""
        # convert all arrays to PyTorch tensors
        x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
        u_train = torch.tensor(u_train, dtype=torch.float32).to(device)
        x_val = torch.tensor(x_val, dtype=torch.float32).to(device)
        u_val = torch.tensor(u_val, dtype=torch.float32).to(device)

        best_rel = 1e9  # best validation error so far

        for epoch in range(1,epochs+1):
            self.opt.zero_grad()

            # ---- Collocation points for physics loss ----
            # randomly sample inside [min,max] range of training data
            lb = torch.min(x_train, 0).values
            ub = torch.max(x_train, 0).values
            x_f = lb + (ub - lb) * torch.rand((colloc, x_train.shape[1]), device=device)

            # ---- Compute physics and data losses ----
            if a_fn is None:
                f = self.pde_fn(self.net, x_f, self.nu)
            else:
                f = self.pde_fn(self.net, a_fn, x_f)
            loss_p = self.loss_fn(f, torch.zeros_like(f))       # PDE residual loss
            u_pred = self.net(x_train)
            loss_d = self.loss_fn(u_pred, u_train)              # supervised data loss
            loss = self.lam_d * loss_d + self.lam_p * loss_p    # total weighted loss

            # ---- Backpropagation ----
            loss.backward()
            self.opt.step()

            # ---- Validation every few steps ----
            if epoch % 500 == 0 or epoch == epochs - 1:
                self.net.eval()
                with torch.no_grad():
                    u_pred_val = self.net(x_val).cpu().numpy()
                    u_val_np = u_val.detach().cpu().numpy() if torch.is_tensor(u_val) else u_val
                rel = np.linalg.norm(u_pred_val - u_val_np) / np.linalg.norm(u_val_np)
                print(f"Epoch {epoch:5d} | Total {loss.item():.6e} | Data {loss_d.item():.3e} | Phys {loss_p.item():.3e} | ValRel {rel:.3e}")
                # save best model
                if rel < best_rel:
                    best_rel = rel
                    torch.save(self.net.state_dict(), "best_model.pt")
        print(f"Best Validation Relative Error: {best_rel}")

    def predict(self, x):
        """Predict using trained network"""
        x = torch.tensor(x, dtype=torch.float32).to(device)
        self.net.eval()
        with torch.no_grad():
            return self.net(x).cpu().numpy()

In [32]:
def load_burgers(path, sample_size=10000):
    """
    Loads Burgers' dataset (.mat file) safely and efficiently.

    - Handles both 2D (single sample) and 3D (multiple samples)
    - Uses only the first `sample_size` samples (default 10,000)
    - Returns flattened arrays X (x,t) and U (u)
    """
    data = loadmat(path)
    u = data["u"]

    # Make sure u is 3D → (N, nt, nx)
    if u.ndim == 2:
        u = u[None, :, :]  # single sample
    if u.shape[1] > u.shape[2]:  # transpose if needed
        u = np.transpose(u, (0, 2, 1))

    # Extract dimensions
    n, nt, nx = u.shape
    x = np.linspace(0, 1, nx)
    t = np.linspace(0, 1, nt)
    X, T = np.meshgrid(x, t)
    X_flat, T_flat = X.ravel(), T.ravel()

    # Flatten into (x, t, u)
    X_all, U_all = [], []
    for i in range(n):
        Ui = u[i].ravel()[:, None]
        Xi = np.stack([X_flat, T_flat], axis=1)
        X_all.append(Xi)
        U_all.append(Ui)

    X_all = np.vstack(X_all)
    U_all = np.vstack(U_all)

    # Subsample to first `sample_size` points
    if X_all.shape[0] > sample_size:
        X_all = X_all[:sample_size]
        U_all = U_all[:sample_size]
    return X_all, U_all


In [33]:
mode = "burgers"
data_path = "/kaggle/input/pde-dataset/burgers_data_R10.mat"
epochs = 5000    # number of training epochs

# Load Burgers dataset
X, U = load_burgers(data_path, sample_size=10000)

# Split into train/validation sets
X_tr, X_val, U_tr, U_val = train_test_split(X, U, test_size=0.2, random_state=42)

In [34]:
# Normalize for stability
xm = np.nanmean(X_tr, axis=0)
xs = np.nanstd(X_tr, axis=0) + 1e-8
um = np.nanmean(U_tr, axis=0)
us = np.nanstd(U_tr, axis=0) + 1e-8

# Replace any NaNs or infs
X_tr = np.nan_to_num((X_tr - xm) / xs)
X_val = np.nan_to_num((X_val - xm) / xs)
U_tr = np.nan_to_num((U_tr - um) / us)
U_val = np.nan_to_num((U_val - um) / us)

In [35]:
# Create and train model
model = PINN(pde_fn=burgers_pde, inp=2, out=1)
model.train(X_tr, U_tr, X_val, U_val, epochs=epochs)

Epoch   500 | Total 4.017894e-02 | Data 7.624e-03 | Phys 3.255e-01 | ValRel 7.689e-02
Epoch  1000 | Total 3.983972e-02 | Data 6.353e-03 | Phys 3.349e-01 | ValRel 7.763e-02
Epoch  1500 | Total 3.780898e-02 | Data 5.953e-03 | Phys 3.186e-01 | ValRel 7.807e-02
Epoch  2000 | Total 3.756800e-02 | Data 5.407e-03 | Phys 3.216e-01 | ValRel 7.966e-02
Epoch  2500 | Total 3.547914e-02 | Data 5.567e-03 | Phys 2.991e-01 | ValRel 7.180e-02
Epoch  3000 | Total 3.627674e-02 | Data 5.330e-03 | Phys 3.095e-01 | ValRel 7.468e-02
Epoch  3500 | Total 3.751700e-02 | Data 6.216e-03 | Phys 3.130e-01 | ValRel 8.758e-02
Epoch  4000 | Total 2.638477e-02 | Data 4.821e-03 | Phys 2.156e-01 | ValRel 6.908e-02
Epoch  4500 | Total 2.023248e-02 | Data 5.358e-03 | Phys 1.487e-01 | ValRel 6.777e-02
Epoch  4999 | Total 2.045881e-02 | Data 4.130e-03 | Phys 1.633e-01 | ValRel 6.233e-02
Epoch  5000 | Total 2.304295e-02 | Data 3.929e-03 | Phys 1.911e-01 | ValRel 6.801e-02
Best Validation Relative Error: 0.06232757121324539
